# Numpy

### Импорт библиотеки

In [2]:
import numpy as np

### Получить датасет Ирис

In [4]:
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data"
data = np.genfromtxt(url, delimiter=",", dtype=str)
print(iris[:10])

<class 'FileNotFoundError'>: https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data not found.

### Представить датасет одномерный/двумерный массивы (признаки поместить в матрицу (n, 4), названия самого ириса --- вектор размера n)
### Отобразить сколько места он занимает

### Нормализовать данные на промежуток от 0 до 1 для каждой колонки

### Представьте один из признаков в виде категориальной переменной и её на три типа используя квантили:
< 0.25 --- small
0.25-0.75 --- medium
> 0.75 --- big

### Разделите датасет на две случайные выборки (0.8 / 0.2)

### Выберите один из методов классификации (к примеру, SVC), обучите на тренировочном датасете, оцените результат на тестовом и сделайте выводы

### Проведите три эксперимента изменяя гиперпараметры модели или условия препроцессинга данных (например, нормализованные или нет данные), сделайте выводы

### Пользуясь методами уменьшения размерности (PCA, t-sne) визуализируйте датасет. Это должен быть scatter plot, где точка это отдельный ирис, ее координаты -- признаки со сжатой выбранной вам размерностью, а цвет -- тип ириса. Нарисуйте этот график дважды, где
Тип ириса выбирается из оригинальных данных
Тип ириса выбирается из предсказанных данных